In [1]:
import sys
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import optuna
import xgboost as xgb
import lightgbm as lgbm
import statistics
from sklearn.linear_model import Ridge
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer
from functools import partial
from openfe import OpenFE, transform
import warnings

warnings.filterwarnings('ignore')


In [2]:
sys.path.append(os.path.abspath("/home/bk_anupam/code/ML/ML_UTILS/"))

In [3]:
import train_tabular_utils as tt
import cv_split_utils
import enums
import data_utils

In [4]:
class Config:
    RUN_MODE = "LOCAL"
    RANDOM_SEED = 42
    NUM_FOLDS = 5
    TARGET_COL_NAME = "Rings"    
    SKEW_THRESHOLD = 0.5
    EARLY_STOPPING = 500
    RESULTS_FILE = "model_execution_results.pkl"
    MODEL_TYPE = enums.ModelName.XGBoost
    REMOVE_OUTLIERS = True
    POWER_TRANSFORM = False
    NORMALIZE_DATA = True
    SCALER = enums.Scaler.StandardScaler
    METRIC = enums.Metrics.RMSLE
    NUM_TUNING_TRIALS = 25
    TUNE_ON_SINGLE_FOLD = True
    GENERATE_AUTO_FEATURES = True
    NUM_NEW_FEATURES = 20

COLS_TO_LEAVE = ["Rings", "kfold"]
CPU_COUNT = os.cpu_count()

DATA_READPATH = "./data/"
DATA_WRITEPATH = "./data/"
if Config.RUN_MODE == "KAGGLE":
    DATA_READPATH = "/kaggle/input/playground-series-s4e4/"
    DATA_WRITEPATH = "/kaggle/working/"

In [5]:
# import train dataset locally from data folder
df_train = pd.read_csv(DATA_READPATH + "train.csv")
# import test dataset locally from data folder
df_test = pd.read_csv(DATA_READPATH + "test.csv")
# drop id column
df_train = df_train.drop("id", axis=1)
df_test = df_test.drop("id", axis=1)
# keep a copy of original train and test data for later use
df_train_orig = df_train.copy()
df_test_orig = df_test.copy()

In [6]:
feature_cols_for_fe = [x for x in df_train.columns if x not in COLS_TO_LEAVE]

In [7]:
def generate_new_features(df_train, df_test, feature_cols, NUM_NEW_FEATURES=10):
    train_X = df_train[feature_cols] 
    test_X = df_test[feature_cols]   
    train_y = df_train[Config.TARGET_COL_NAME]
    ofe = OpenFE()
    features = ofe.fit(data=train_X, label=train_y, n_jobs=CPU_COUNT, verbose=False)  # generate new features    
    # OpenFE recommends a list of new features. We include the top 10
    # generated features to see how they influence the model performance
    train_X, test_X = transform(train_X, test_X, ofe.new_features_list[:NUM_NEW_FEATURES], n_jobs=CPU_COUNT)        
    return train_X, test_X, features

In [8]:
df_train, df_test, new_features = generate_new_features(df_train, df_test, feature_cols_for_fe, Config.NUM_NEW_FEATURES)  
df_train_labels = df_train_orig[[Config.TARGET_COL_NAME]]
# Add the label data to the dataframe
df_train = pd.concat([df_train, df_train_labels], axis=1)
# save the new train and test data with openfe features to csv files for later use
df_train.to_csv(DATA_WRITEPATH + "train_openfe.csv", index=False)
df_test.to_csv(DATA_WRITEPATH + "test_openfe.csv", index=False)

  0%|          | 0/30 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000155 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 125
[LightGBM] [Info] Number of data points in the train set: 9061, number of used features: 1
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000170 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 9061, number of used features: 1
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000183 seconds.
You can set `force_col_wise=true` to remove the overhead.[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000144 seconds.
You can set `force_col_wise=true` to remove the overhead.

[LightGBM] [Info] Total Bins 255[LightGBM] [Info] Total Bins 47
[LightGBM] [Info] Number of data points 

  3%|▎         | 1/30 [00:02<01:06,  2.29s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf[LightGBM] [Warning] No further splits with positive gain, best gain: -inf



[LightGBM] [Warning] No further splits with positive gain, best gain: -inf[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf[LightGBM] [Warning] No further splits with positive gain, best gain: -inf[LightGBM] [Warning] No 

 10%|█         | 3/30 [00:03<00:24,  1.08it/s]

...

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000775 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 234
[LightGBM] [Info] Number of data points in the train set: 72492, number of used features: 1
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000611 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 58
[LightGBM] [Info] Number of data points in the train set: 72492, number of used features: 1


 97%|█████████▋| 31/32 [00:34<00:00,  1.08it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000663 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 56
[LightGBM] [Info] Number of data points in the train set: 72492, number of used features: 1
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000627 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 72492, number of used features: 1


100%|██████████| 32/32 [00:17<00:00,  1.87it/s]


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.053437 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 38955
[LightGBM] [Info] Number of data points in the train set: 72492, number of used features: 195
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines


In [9]:
print(f"Number of new features = {len(new_features)}")  
print(new_features)

Number of new features = 188
[<openfe.FeatureGenerator.Node object at 0x7f8958e71d30>, <openfe.FeatureGenerator.Node object at 0x7f8958e88040>, <openfe.FeatureGenerator.Node object at 0x7f8958e77580>, <openfe.FeatureGenerator.Node object at 0x7f8958e8a820>, <openfe.FeatureGenerator.Node object at 0x7f8958e75f70>, <openfe.FeatureGenerator.Node object at 0x7f8958e88100>, <openfe.FeatureGenerator.Node object at 0x7f8958e7e6a0>, <openfe.FeatureGenerator.Node object at 0x7f8958e8a190>, <openfe.FeatureGenerator.Node object at 0x7f8951f7fd30>, <openfe.FeatureGenerator.Node object at 0x7f8958e8a610>, <openfe.FeatureGenerator.Node object at 0x7f8951feaf40>, <openfe.FeatureGenerator.Node object at 0x7f8951f8c8b0>, <openfe.FeatureGenerator.Node object at 0x7f8958e7cdf0>, <openfe.FeatureGenerator.Node object at 0x7f8958061160>, <openfe.FeatureGenerator.Node object at 0x7f8958061d90>, <openfe.FeatureGenerator.Node object at 0x7f8958e8a8e0>, <openfe.FeatureGenerator.Node object at 0x7f8958e77730>, <

In [10]:
df_train.head()

,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,autoFE_f_0,autoFE_f_1,...,autoFE_f_11,autoFE_f_12,autoFE_f_13,autoFE_f_14,autoFE_f_15,autoFE_f_16,autoFE_f_17,autoFE_f_18,autoFE_f_19,Rings
0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,2.291667,1.368750,...,-0.0935,0.2400,0.575,3.666667,0.557356,0.175106,5.266212,0.7715,0.249180,11
1,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,1.968750,1.431250,...,-0.0435,0.3200,0.585,4.344828,0.433628,0.175106,4.086799,0.1300,0.567989,11
2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,32.000000,1.100000,...,-0.0020,0.0250,0.185,6.400000,5.238095,0.117992,7.000000,0.0210,0.003235,6
3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,2.380000,1.502000,...,-0.0445,0.2500,0.575,3.966667,0.519410,0.184416,4.450122,0.9145,0.416416,10
4,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,2.810127,1.870886,...,-0.0375,0.1975,0.585,4.269231,0.543478,0.117992,4.887500,0.7820,0.937733,9
